In [1]:
import numpy as np
import pandas as pd
import random
import math

In [2]:
ev = pd.read_csv("../Desktop/Electric Vehicles.csv")
cs = pd.read_csv("../Desktop/Charging Stations.csv")
ad = pd.read_csv("../Desktop/Addresses.csv", low_memory=False)

In [3]:
ev = ev[ev["State"] == "WA"]
ev = ev[ev["City"] == "Seattle"]
print(len(ev))

29447


In [4]:
cs = cs[cs["City"] == "Seattle"]
cs = cs[cs["Access Days Time"] == "24 hours daily"]
cs = cs.drop(cs[cs["Restricted Access"] == True].index)
cs['Plugs'] = cs["EV Level2 EVSE Num"].fillna(0) + cs["EV DC Fast Count"].fillna(0)

In [5]:
ev["Postal Code"] = ev["Postal Code"].dropna().astype(int).astype(str)

pc_count = ev.groupby("Postal Code")["VIN (1-10)"].count()

wa_pcs = ad["POSTCODE"].dropna().unique().astype(str)

sampled_coordinates = pd.DataFrame()

for pc in pc_count.index:
    if pc in wa_pcs:
        pc_coords = ad.groupby("POSTCODE")[["LON", "LAT"]].get_group(pc)
        if len(pc_coords) > 0:
            ev_coords = pc_coords.sample(n=pc_count[pc], replace=True, random_state=42)  
            sampled_coordinates = pd.concat([sampled_coordinates, ev.groupby("Postal Code").get_group(pc).assign(LON = ev_coords["LON"].values, LAT = ev_coords["LAT"].values)], ignore_index=True)
display(sampled_coordinates)

,VIN (1-10),County,City,State,Postal Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,2020 Census Tract,LON,LAT
0,5YJ3E1EA6P,King,Seattle,WA,98055,2023,TESLA,MODEL 3,Battery Electric Vehicle (BEV),Eligibility unknown as battery range has not b...,0,0,11.0,227192206,POINT (-122.197 47.43876),CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),5.303303e+10,-122.190667,47.452596
1,5YJ3E1EA3K,King,Seattle,WA,98055,2019,TESLA,MODEL 3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,220,0,11.0,257251753,POINT (-122.197 47.43876),CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),5.303303e+10,-122.191863,47.438811
2,KNDJX3AE4J,King,Seattle,WA,98055,2018,KIA,SOUL EV,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,111,33950,37.0,169857530,POINT (-122.197 47.43876),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),5.303303e+10,-122.197307,47.482419
3,JTMABABA0P,King,Seattle,WA,98055,2023,SUBARU,SOLTERRA,Battery Electric Vehicle (BEV),Eligibility unknown as battery range has not b...,0,0,37.0,233864046,POINT (-122.197 47.43876),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),5.303303e+10,-122.197648,47.473818
4,5YJSA1E40L,King,Seattle,WA,98055,2020,TESLA,MODEL S,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,337,0,37.0,230134271,POINT (-122.197 47.43876),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),5.303303e+10,-122.212967,47.460819
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29442,JHMZC5F30J,King,Seattle,WA,98199,2018,HONDA,CLARITY,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,47,0,36.0,331957342,POINT (-122.394185 47.639195),CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),5.303301e+10,-122.391736,47.637859
29443,1V2GNPE84P,King,Seattle,WA,98199,2023,VOLKSWAGEN,ID.4,Battery Electric Vehicle (BEV),Eligibility unknown as battery range has not b...,0,0,36.0,220480375,POINT (-122.394185 47.639195),CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),5.303301e+10,-122.401249,47.638032
29444,7FCTGAAAXN,King,Seattle,WA,98199,2022,RIVIAN,R1T,Battery Electric Vehicle (BEV),Eligibility unknown as battery range has not b...,0,0,36.0,225874211,POINT (-122.394185 47.639195),CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),5.303301e+10,-122.393663,47.632505
29445,7SAYGDEF2N,King,Seattle,WA,98199,2022,TESLA,MODEL Y,Battery Electric Vehicle (BEV),Eligibility unknown as battery range has not b...,0,0,36.0,192823501,POINT (-122.394185 47.639195),CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),5.303301e+10,-122.388100,47.653048


In [6]:
print(len(ev))

29447


In [7]:
def haversine(lat1, lon1, lat2, lon2):
    R = 3958.8
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad
    a = math.sin(dlat/2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c
    return distance

In [8]:
def distance_matrix(lat1, lon1, lat2, lon2):
    distance = np.zeros((len(lat1), len(lat2)))
    for i in range(len(lat1)):
        for j in range(len(lat2)):
            distance[i, j] = haversine(lon1[i], lat1[i], lon2[j], lat2[j])
    return distance

In [9]:
lat1 = sampled_coordinates["LAT"].tolist()
lon1 = sampled_coordinates["LON"].tolist()
lat2 = cs["Latitude"].tolist()
lon2 = cs["Longitude"].tolist()

In [10]:
d = distance_matrix(lat1, lon1, lat2, lon2)
print(d.shape)

(29447, 480)


In [11]:
d_i = d.min(axis=1)

In [12]:
d_i.sum()

37955.961273564055

In [13]:
cs_oos = pd.read_csv("../Desktop/Out-of-Service Charging Stations.csv")

In [14]:
cs_oos = cs_oos[cs_oos["City"] == "Seattle"]
cs_oos = cs_oos[cs_oos["Access Days Time"] == "24 hours daily"]
cs_oos = cs_oos.drop(cs_oos[cs_oos["Restricted Access"] == True].index)
cs_oos["Plugs"] = cs_oos["EV Level2 EVSE Num"] + cs_oos["EV DC Fast Count"]

In [15]:
print(len(cs_oos))

84


In [16]:
lat1_oos = sampled_coordinates["LAT"].tolist()
lon1_oos = sampled_coordinates["LON"].tolist()
lat2_oos = cs_oos["Latitude"].tolist()
lon2_oos = cs_oos["Longitude"].tolist()

In [17]:
d_ij = distance_matrix(lat1_oos, lon1_oos, lat2_oos, lon2_oos)
print(d_ij.shape)

(29447, 84)


In [18]:
import gurobipy as gp
from gurobipy import GRB

n = len(sampled_coordinates)
m = len(cs_oos)
d_i = d_i
d_ij = d_ij

model = gp.Model("ev_cs")

x = {}
for i in range(n):
    for j in range(m):
        x[i, j] = model.addVar(vtype=GRB.BINARY, name=f"x[{i},{j}]")

y = {}
for j in range(m):
    y[j] = model.addVar(vtype=GRB.BINARY, name=f"y[{j}]")

model.setObjective((gp.quicksum((d_i[i] * x[i, j]) - (d_ij[i][j] * x[i, j]) for j in range(m) for i in range(n))), sense=GRB.MAXIMIZE)

for i in range(n):
    model.addConstr(gp.quicksum(x[i, j] for j in range(m)) <= 1)
    
for i in range(n):
    for j in range(m):
        model.addConstr(d_ij[i][j] * x[i, j] <= d_i[i])
    
for i in range(n):
    for j in range(m):
        model.addConstr(x[i, j] <= y[j])
        
model.addConstr(gp.quicksum(y[j] for j in range(m)) <= 0.5*m) # figure out less arbitrary constraint than 50%
        
model.optimize()

for j in range(m):
    if y[j].x > 0.5:
        print(f"charging station {j} is reopened")
        
for i in range(n):
    for j in range(m):
        if x[i, j].x > 0.5:
            print(f"electric vehicle {i} goes to charging station {j}")

Set parameter Username
Academic license - for non-commercial use only - expires 2025-04-11
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[x86] - Darwin 21.6.0 21G217)

CPU model: Intel(R) Core(TM) i5-5350U CPU @ 1.80GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 4976544 rows, 2473632 columns and 9894276 nonzeros
Model fingerprint: 0xd6f46484
Variable types: 0 continuous, 2473632 integer (2473632 binary)
Coefficient statistics:
  Matrix range     [9e-04, 2e+02]
  Objective range  [3e-06, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e-05, 2e+02]
Found heuristic solution: objective -0.0000000
Presolve removed 4962885 rows and 2462356 columns
Presolve time: 3.22s
Presolved: 13659 rows, 11276 columns, 29023 nonzeros
Variable types: 0 continuous, 11276 integer (11276 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.6648679e+03   2.266000e+03   0.000000e

electric vehicle 559 goes to charging station 21
electric vehicle 560 goes to charging station 27
electric vehicle 564 goes to charging station 21
electric vehicle 565 goes to charging station 27
electric vehicle 566 goes to charging station 27
electric vehicle 567 goes to charging station 27
electric vehicle 571 goes to charging station 21
electric vehicle 572 goes to charging station 21
electric vehicle 573 goes to charging station 21
electric vehicle 574 goes to charging station 21
electric vehicle 578 goes to charging station 27
electric vehicle 586 goes to charging station 21
electric vehicle 590 goes to charging station 27
electric vehicle 593 goes to charging station 21
electric vehicle 595 goes to charging station 21
electric vehicle 599 goes to charging station 21
electric vehicle 605 goes to charging station 27
electric vehicle 606 goes to charging station 21
electric vehicle 615 goes to charging station 59
electric vehicle 618 goes to charging station 21
electric vehicle 624

electric vehicle 2082 goes to charging station 8
electric vehicle 2085 goes to charging station 26
electric vehicle 2086 goes to charging station 71
electric vehicle 2087 goes to charging station 8
electric vehicle 2089 goes to charging station 15
electric vehicle 2090 goes to charging station 8
electric vehicle 2092 goes to charging station 8
electric vehicle 2110 goes to charging station 8
electric vehicle 2112 goes to charging station 8
electric vehicle 2116 goes to charging station 8
electric vehicle 2117 goes to charging station 26
electric vehicle 2118 goes to charging station 8
electric vehicle 2122 goes to charging station 15
electric vehicle 2123 goes to charging station 71
electric vehicle 2124 goes to charging station 8
electric vehicle 2127 goes to charging station 8
electric vehicle 2130 goes to charging station 15
electric vehicle 2133 goes to charging station 26
electric vehicle 2135 goes to charging station 15
electric vehicle 2137 goes to charging station 8
electric ve

electric vehicle 2560 goes to charging station 71
electric vehicle 2564 goes to charging station 8
electric vehicle 2566 goes to charging station 8
electric vehicle 2572 goes to charging station 8
electric vehicle 2574 goes to charging station 8
electric vehicle 2578 goes to charging station 15
electric vehicle 2588 goes to charging station 8
electric vehicle 2591 goes to charging station 8
electric vehicle 2594 goes to charging station 26
electric vehicle 2596 goes to charging station 8
electric vehicle 2597 goes to charging station 8
electric vehicle 2600 goes to charging station 8
electric vehicle 2608 goes to charging station 15
electric vehicle 2610 goes to charging station 8
electric vehicle 2613 goes to charging station 15
electric vehicle 2614 goes to charging station 26
electric vehicle 2615 goes to charging station 8
electric vehicle 2619 goes to charging station 8
electric vehicle 2625 goes to charging station 8
electric vehicle 2627 goes to charging station 8
electric vehic

electric vehicle 3114 goes to charging station 26
electric vehicle 3115 goes to charging station 8
electric vehicle 3123 goes to charging station 8
electric vehicle 3126 goes to charging station 8
electric vehicle 3130 goes to charging station 71
electric vehicle 3131 goes to charging station 8
electric vehicle 3132 goes to charging station 8
electric vehicle 3134 goes to charging station 26
electric vehicle 3139 goes to charging station 26
electric vehicle 3141 goes to charging station 8
electric vehicle 3144 goes to charging station 39
electric vehicle 3148 goes to charging station 8
electric vehicle 3150 goes to charging station 8
electric vehicle 3151 goes to charging station 8
electric vehicle 3159 goes to charging station 8
electric vehicle 3162 goes to charging station 8
electric vehicle 3163 goes to charging station 15
electric vehicle 3167 goes to charging station 8
electric vehicle 3168 goes to charging station 15
electric vehicle 3169 goes to charging station 8
electric vehi

...

electric vehicle 23112 goes to charging station 44
electric vehicle 23113 goes to charging station 44
electric vehicle 23114 goes to charging station 44
electric vehicle 23116 goes to charging station 81
electric vehicle 23117 goes to charging station 44
electric vehicle 23118 goes to charging station 44
electric vehicle 23120 goes to charging station 44
electric vehicle 23121 goes to charging station 44
electric vehicle 23123 goes to charging station 44
electric vehicle 23124 goes to charging station 81
electric vehicle 23125 goes to charging station 44
electric vehicle 23126 goes to charging station 44
electric vehicle 23130 goes to charging station 44
electric vehicle 23131 goes to charging station 44
electric vehicle 23133 goes to charging station 44
electric vehicle 23134 goes to charging station 81
electric vehicle 23135 goes to charging station 44
electric vehicle 23138 goes to charging station 44
electric vehicle 23142 goes to charging station 44
electric vehicle 23143 goes to 

electric vehicle 23878 goes to charging station 40
electric vehicle 23879 goes to charging station 83
electric vehicle 23880 goes to charging station 40
electric vehicle 23881 goes to charging station 26
electric vehicle 23882 goes to charging station 39
electric vehicle 23885 goes to charging station 8
electric vehicle 23888 goes to charging station 26
electric vehicle 23892 goes to charging station 16
electric vehicle 23895 goes to charging station 39
electric vehicle 23896 goes to charging station 40
electric vehicle 23897 goes to charging station 40
electric vehicle 23898 goes to charging station 8
electric vehicle 23900 goes to charging station 8
electric vehicle 23906 goes to charging station 39
electric vehicle 23909 goes to charging station 26
electric vehicle 23910 goes to charging station 26
electric vehicle 23914 goes to charging station 8
electric vehicle 23915 goes to charging station 8
electric vehicle 23917 goes to charging station 40
electric vehicle 23918 goes to charg

electric vehicle 24651 goes to charging station 81
electric vehicle 24668 goes to charging station 44
electric vehicle 24669 goes to charging station 44
electric vehicle 24680 goes to charging station 44
electric vehicle 24710 goes to charging station 44
electric vehicle 24742 goes to charging station 44
electric vehicle 24767 goes to charging station 44
electric vehicle 24784 goes to charging station 44
electric vehicle 24794 goes to charging station 44
electric vehicle 24802 goes to charging station 44
electric vehicle 24806 goes to charging station 44
electric vehicle 24813 goes to charging station 44
electric vehicle 24819 goes to charging station 44
electric vehicle 24820 goes to charging station 44
electric vehicle 24830 goes to charging station 44
electric vehicle 24831 goes to charging station 44
electric vehicle 24836 goes to charging station 44
electric vehicle 24837 goes to charging station 44
electric vehicle 24846 goes to charging station 44
electric vehicle 24850 goes to 

electric vehicle 27145 goes to charging station 39
electric vehicle 27150 goes to charging station 39
electric vehicle 27157 goes to charging station 36
electric vehicle 27158 goes to charging station 36
electric vehicle 27161 goes to charging station 39
electric vehicle 27166 goes to charging station 36
electric vehicle 27168 goes to charging station 36
electric vehicle 27172 goes to charging station 39
electric vehicle 27177 goes to charging station 36
electric vehicle 27185 goes to charging station 36
electric vehicle 27189 goes to charging station 36
electric vehicle 27192 goes to charging station 36
electric vehicle 27195 goes to charging station 36
electric vehicle 27198 goes to charging station 36
electric vehicle 27202 goes to charging station 36
electric vehicle 27208 goes to charging station 36
electric vehicle 27214 goes to charging station 36
electric vehicle 27219 goes to charging station 36
electric vehicle 27238 goes to charging station 36
electric vehicle 27240 goes to 

electric vehicle 29036 goes to charging station 11
electric vehicle 29038 goes to charging station 11
electric vehicle 29040 goes to charging station 11
electric vehicle 29041 goes to charging station 11
electric vehicle 29045 goes to charging station 11
electric vehicle 29051 goes to charging station 11
electric vehicle 29053 goes to charging station 11
electric vehicle 29054 goes to charging station 11
electric vehicle 29055 goes to charging station 11
electric vehicle 29058 goes to charging station 11
electric vehicle 29062 goes to charging station 11
electric vehicle 29064 goes to charging station 11
electric vehicle 29070 goes to charging station 11
electric vehicle 29071 goes to charging station 11
electric vehicle 29082 goes to charging station 11
electric vehicle 29086 goes to charging station 11
electric vehicle 29087 goes to charging station 11
electric vehicle 29093 goes to charging station 11
electric vehicle 29098 goes to charging station 11
electric vehicle 29104 goes to 

In [19]:
active_count = 0

for i in range(n):
    for j in range(m):
        if x[i, j].x > 0.5:
            active_count += 1

print(active_count)

6989


In [20]:
active_count = 0

for j in range(m):
    if y[j].x > 0.5:
        active_count += 1

print(active_count)

42


In [21]:
# number of cars at each charging station before vs. after charging stations are reopened
# average distance traveled before vs. after charging stations are reopened
# environmental impact of objective function value NOT traveled over some period of time (need to calculate how often people travel to charge their car)
# limitation: capacity of charging stations; however, a lot of cars going to one charging station demonstrates a higher demand for more charging stations in that area
# changes with sampling; NOT a simulation project but ideally we'd have exact address/coordinate data for each car